# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** The supervised machine learning classication it is the type of supervised learning need here, because we need to classify student to know if need or not an early intervention. If this problem needs a continuous output it would be a regression problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import LogisticRegression
import random
from sklearn import svm
from IPython.display import display

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
n_students = student_data.shape[0] #The total number of students
n_features = student_data.shape[1] - 1 #The total number of features for each student
n_passed = len(student_data.loc[student_data['passed'] == 'yes']) #The number of those students who passed
n_failed = len(student_data.loc[student_data['passed'] == 'no']) #The number of those students who failed
grad_rate = (float(n_passed)/n_students)*100 #grade = passed/students x 100

# ====-----   no code changes ! -=========
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here

# TODO: Set the number of training points
num_train = 300 #approximately 75%

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
#http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=233)
# Show the results of the split
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])

Training set: 300 samples
Test set: 95 samples


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

K Nearest Neighbors: Very robust supervised learning. Gennerally used for classification and regression problems. 
Strengths: Robust when have noisy training data and work very well need to training with a large database, but is the weaknesses are the Computation cost to training, because need to compute distance of each query instance to all training samples, and when the number of features grows, the data grows exponentially. One general applications very been successfully applied is the fault detection in industrial processes, the main reason is because the K Nearest Neighbors is very simple to understand, however works incredibly well.
**Why did you choose this model to apply?:**: I my opinion the K-Nearest Neighbors as one of the possible models that really go well with this kind of data, because it, K-Nearest Neighbors does well generalizing data and membership (based on similarity) and if the evaluation shows close (or falls) into an existing group, the K-Nearest Neighbors can easily clarify it based on distance from other members of its class (I was wrong).

Decision Tree: It is a supervised learning that can be used for classification problems. Strengths: It is a supervised learning very easy to interpret graphically and don't need a normalized values and becuase that is very to use with just a little data preparation. Weaknesses: They can be extremely sensitive to small perturbations in the data: a slight change can result in a drastically different treeand can easily overfit. They can have problems out-of-sample prediction. Decision tree can be used for a lot of things, Earthquake and disaster planning, Structural planning and integration, Portfolio selection, Real options analysis, Job classification, Applicant evaluation and etc. Decision tree learning is a method commonly used in data mining and is a simple representation for classifying examples and can be "learned" by splitting the source set into subsets based on an attribute value test. 
**Why did you choose this model to apply?:**: Because of this kind of data I believed with Decision Tree the data may lend itself well to a bunch of linear seperators, because there is a lot of binary data.

Support Vector Machine: classification, regression and outliers detection.Strengths: SVM is a useful method for
classification and  Much active research is taking place on areas related to SVM. Still effective in cases where number of dimensions is greater than the number of samples. Weaknesses: the optimal design for multiclass SVC classifiers is a further area for research, Although SVMs have good generalization performance, they can be abysmally slow in test phase, a problem addressed in. The training time complexity is O(n³) and because that is very computationally expensive to trainand and finally Need a “good” kernel function. I found many articles about the use of SVM in Exploratory of Petroleum and Natural Gas. "SVM as one of the most promising intelligent techniques for data analysis found in the published literature, as theoretical approaches and sophisticated applications developed for various research areas and problem domains." https://www.researchgate.net/publication/243190076_Applications_of_Support_Vector_Machines_in_the_Exploratory_Phase_of_Petroleum_and_Natural_Gas_a_Survey
**Why did you choose this model to apply?:** I chose this model because the problem that we want to solve is classification. The structure of the data set is a classic case for SVM, each label can be described with a vector of values. One important point is the small data set and because that the training will be done in a short time and seems the SVM will have some advantage with the data set.


** After modification ** 
After, all modification and reviews, the results do not change much. So, because that, my opinion of the algorithms stays the same​.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [10]:
import time
from sklearn.svm import SVC

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.6f}".format(end - start)

#takes as input a classifier and training data and fits the classifier to the data.
svm_clf = SVC()
train_classifier(svm_clf, X_train, y_train)

print " *******************************************  "

from sklearn.metrics import f1_score
def predict_labels(clf, features, target):
    start = time.time()
    y_pred = clf.predict(features)
    print "Done!"
    end = time.time()
    print "Made predictions in {:.6f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(svm_clf, X_train, y_train)

print "F1 score for training set: {}".format(train_f1_score)

print " *******************************************  "

def train_predict(clf, X_train, y_train, X_test, y_test):
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.6f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.6f}.".format(predict_labels(clf, X_test, y_test))
    
    
# TODO: Run the helper function above for desired subsets of training data
for n in (100,200,300):
    train_predict(svm_clf,X_train[:n], y_train[:n], X_test, y_test)
    
    



Training SVC...
Done!
Training time (secs): 0.009271
 *******************************************  
Done!
Made predictions in 0.006319 seconds.
F1 score for training set: 0.861607142857
 *******************************************  
Training a SVC using a training set size of 100. . .
Training SVC...
Done!
Training time (secs): 0.001924
Done!
Made predictions in 0.001033 seconds.
F1 score for training set: 0.876712.
Done!
Made predictions in 0.000965 seconds.
F1 score for test set: 0.853503.
Training a SVC using a training set size of 200. . .
Training SVC...
Done!
Training time (secs): 0.003600
Done!
Made predictions in 0.002389 seconds.
F1 score for training set: 0.859935.
Done!
Made predictions in 0.001245 seconds.
F1 score for test set: 0.846154.
Training a SVC using a training set size of 300. . .
Training SVC...
Done!
Training time (secs): 0.007563
Done!
Made predictions in 0.005078 seconds.
F1 score for training set: 0.861607.
Done!
Made predictions in 0.001650 seconds.
F1 score

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [11]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import LogisticRegression
import time

my_random_state=10

# TODO: Initialize the three models
clf_A = neighbors.KNeighborsClassifier(n_neighbors=10)
clf_B = tree.DecisionTreeClassifier(random_state=my_random_state)
clf_C = svm.SVC(random_state=my_random_state) #Use a random_state for each model you use, if provided.

clfs = [clf_A,clf_B,clf_C]


X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train[0:300]
y_train_300 = y_train[0:300]

train_sets = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]
   

# TODO: Execute the 'train_predict' function for each classifier and each training set size    
for clf in [clf_A, clf_B, clf_C]:
    print "\n{}: \n".format(clf.__class__.__name__)
    for X, y in train_sets:
        train_predict(clf, X, y, X_test, y_test)
        
        
        




KNeighborsClassifier: 

Training a KNeighborsClassifier using a training set size of 100. . .
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001363
Done!
Made predictions in 0.001625 seconds.
F1 score for training set: 0.832117.
Done!
Made predictions in 0.002160 seconds.
F1 score for test set: 0.733813.
Training a KNeighborsClassifier using a training set size of 200. . .
Training KNeighborsClassifier...
Done!
Training time (secs): 0.000819
Done!
Made predictions in 0.004012 seconds.
F1 score for training set: 0.826389.
Done!
Made predictions in 0.002668 seconds.
F1 score for test set: 0.732394.
Training a KNeighborsClassifier using a training set size of 300. . .
Training KNeighborsClassifier...
Done!
Training time (secs): 0.000940
Done!
Made predictions in 0.006988 seconds.
F1 score for training set: 0.819905.
Done!
Made predictions in 0.002234 seconds.
F1 score for test set: 0.794326.

DecisionTreeClassifier: 

Training a DecisionTreeClassifier using a training set

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - KNeighborsClassifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.001625              |0.002160                | 0.832117         | 0.733813        |
| 200               |  0.004012              |  0.002668              |  0.826389        | 0.732394        |
| 300               |             0.006988    |         0.002234       |  0.819905        |  0.794326       |

** Classifer 2 - DecisionTreeClassifier **

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.000290           |          0.000248      |     1.000000     |    0.677165     |
| 200               |    0.000205           |     0.000181           |    1.000000      |    0.702290     |
| 300               |         0.000223        |    0.000182            |   1.000000       |  0.751880       |

** Classifer 3 - Support Vector Machine **

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.000797              |             0.000755   |          0.876712|      0.853503   |
| 200               |    0.002430             |  0.001220              |        0.859935  |      0.846154   |
| 300               |           0.009598      |          0.002862      |       0.861607   |      0.836601   | 

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** Based on the experiments, the best model to use is SVM. The SVM have almost the same performance for different training data and especialy for the last one (300), this is a strong indicator that the size of the training data is not the most important thing here. Finally the SVG gets the best F1 scores. I believe the performance worth the cost.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: ** When compare the performance of all three candidate algorithms, Support vector machines get the highest F1 scores on the test set. If the use of support vector machines is not prohibitively high, it seems to be the best choice. The training data can be "only" used the infomation of initially and the end of each school year. The training time take just a small annual cost and it will could without significant impact. 

The SVM is very robust to very large number of variables and small samples and can learn both simple and highly complex classification models. The goal of the SVM is to train a model that assigns new unseen objects into a particular category. It achieves this by creating a linear partition of the feature space into two categories, formally defined by a separating hyperplane (figure 1 and 2) and will learn from data you give it on past students and whether they passed or failed, and this algorithms is trained by trying to find a linear separator between the positive test cases and the negative tests cases and the SVM try to maximize the margin width around this linear separator and the points close to the margin are the support vectors, with these vectors the SVM algorithm define the model. After the model is constructed, there is no need to store the remaining points and hence the model is small.

![figure 1](https://s3.amazonaws.com/quantstart/media/images/qs-svm-0001.png)

![figure 2](https://s3.amazonaws.com/quantstart/media/images/qs-svm-0002.png)


However, much of the benefit of SVMs comes from the fact that they are not restricted to being linear classifiers(some cases, there may appear to be no linear separator). Utilising a technique known as the kernel trick they can become much more flexible by introducing various types of non-linear decision boundaries.

Another important point is the use of Memory Efficiency: "Since only a subset of the training points are used in the actual decision process of assigning new members, only these points need to be stored in memory (and calculated upon) when making decisions" https://www.quantstart.com/articles/Support-Vector-Machines-A-Guide-for-Beginners

So, after all observation the SVM is very robust, have the best performance, the impact of his use is small (computacional and financial) and get the best F1 scores, using all this observation in my opinion the SVM is the best choice. 

Some extra information used by me to defini the beneficitys of SVM.
https://www.quantstart.com/articles/Support-Vector-Machines-A-Guide-for-Beginners
http://www.svms.org/tutorials/Berwick2003.pdf
http://www.med.nyu.edu/chibi/sites/default/files/chibi/Final.pdf





### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [9]:

# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler

# TODO: Create the parameters list you wish to tune
C_range = np.logspace(-1, 3, 4,8,10)
gamma_range = np.logspace(-1, 3, 4,8,10)

parameters = [{'C' : C_range}, {'gamma' : gamma_range}]
# TODO: Initialize the classifier
scaler = StandardScaler()
svr = svm.SVC(random_state=10)
clf = GridSearchCV(svr, parameters)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label = 'yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(svr, parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = clf.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))


Done!
Made predictions in 0.004784 seconds.
Tuned model has a training F1 score of 0.9065.
Done!
Made predictions in 0.002143 seconds.
Tuned model has a testing F1 score of 0.8108.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The Best F1 score value (After redoing the grid search) tuned was 0.9065 and is (still) very high from the best value of to untuned model. This information is very important to see how important is to find best values to tune any algorithm.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.